## Ignore Warning

In [1]:
import warnings
warnings.filterwarnings('ignore')

## Import API

In [2]:
from utils import load_env
load_env()
from utils import get_tavily_api_key
TAVILY_API_KEY = get_tavily_api_key()

## Import Function

In [3]:
from utils import llama32
from utils import llama31
from utils import cprint
from utils import disp_image
import json

## User Interface

In [11]:
import gradio as gr
from PIL import Image
import os

# Define the folder where your images are stored
IMAGE_FOLDER = "images"

# Function to process the uploaded image
def process_image(image):
    # Save the uploaded image to the "images" folder
    image_path = os.path.join(IMAGE_FOLDER, "uploaded_image.jpg")
    image.save(image_path)
    
    # Open and return the image
    img = Image.open(image_path)
    return img

# Create Gradio interface
interface = gr.Interface(
    fn=process_image,              # The function to process the image
    inputs=gr.Image(type="pil"),   # Accept image input in PIL format
    outputs=gr.Image(type="pil"),  # Display the output image
    title="Upload and Process Image",
    description="Upload an image and it will be saved in the images folder."
)

# Launch the interface in the notebook
interface.launch(share=True, inline=True)


Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://1debdc491e4d3e03c9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [12]:
import base64

def encode_image(relative_image_path):
    # Get the absolute path relative to the notebook location
    base_path = os.getcwd()  # Get current working directory
    image_path = os.path.join(base_path, relative_image_path)

    # Encode the image
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

# Provide the relative path to the image
relative_path = "images/uploaded_image.jpg"
base64_image = encode_image(relative_path)


## Prompt Engineering

In [13]:
def llama32pi(prompt, image_url, model_size=11):
  messages = [
    {
      "role": "user",
      "content": [
        {"type": "text",
          "text": prompt},
        {"type": "image_url",
          "image_url": {
            "url": image_url}
        }
      ]
    },
  ]

  result = llama32(messages, model_size)
  return result

In [21]:
question = ("As a professional nutritionist and food detection expert, analyze the image provided of the interior of a refrigerator.You are analyzing the contents of a typical household refrigerator to assist in meal planning, grocery shopping, or nutritional advice. The goal is to provide a clear, concise, and comprehensive inventory of the food items present. Identify and list all visible food ingredients in the refrigerator, specifying details such as the type of food (e.g., fruits, vegetables, dairy, proteins), packaging (e.g., jar, container, loose), and the condition or freshness of each item if possible. Prioritize accuracy and clarity in your response.")

In [23]:
base64_image = encode_image("images/uploaded_image.jpg")
result = llama32pi(question, f"data:image/jpg;base64,{base64_image}")
print(result)

The refrigerator contains a variety of food items, including:

**Dairy and Eggs**

* A container of Becel margarine
* A container of yogurt
* A carton of milk
* A dozen eggs

**Meat and Poultry**

* A package of ground beef
* A package of chicken breasts
* A package of pork chops

**Fruits and Vegetables**

* A head of lettuce
* A bunch of celery
* A bag of apples
* A bag of oranges
* A bag of grapes
* A bag of carrots
* A bag of broccoli

**Beverages**

* A bottle of juice
* A can of soda
* A carton of juice
* A bottle of water

**Condiments and Sauces**

* A jar of mayonnaise
* A jar of ketchup
* A bottle of hot sauce
* A bottle of soy sauce

**Miscellaneous**

* A bag of ice
* A box of cereal
* A bag of coffee
* A box of tea

Overall, the refrigerator appears to be well-stocked with a variety of food items, including dairy products, meats, fruits, vegetables, beverages, condiments, and miscellaneous items. The condition and freshness of each item are not explicitly stated, but based

## Search API

In [2]:
class Conversation:
    def __init__(self, system=""):
        self.messages = []
        if system:
            self.messages.append({"role": "system", "content": system})
    def generate(self, user_question, model=8, temp=0):
        self.messages.append({"role": "user", "content":user_question})
        response = llama31(self.messages, model, temperature=temp)
        self.messages.append({"role":"assistant", "content":response})
        return response
        

In [28]:
import gradio as gr
from tavily import TavilyClient
tavily_client = TavilyClient(api_key=TAVILY_API_KEY)

# Define function to use Tavily API to search for food-related information
def search_food_information(food_item):
    search_result = tavily_client.search("Describe nutrition of" + food_item)
    return search_result

# Gradio interface to take input and display the result
search_interface = gr.Interface(
    fn=search_food_information,
    inputs="text",
    outputs="json",
    title="Food Information Search",
    description="Enter a food item to get more information using Tavily."
)

# Launch the interface in the notebook
search_interface.launch(share=True, inline=True)


Running on local URL:  http://127.0.0.1:7863
Running on public URL: https://ccfd0bdfea31bf0e18.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
